In [1]:
from bs4 import BeautifulSoup
import requests
import json
import re


In [2]:
Website_name='Mondamin'
Website_url='https://www.mondamin.com/'
HEADERS =({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
                   'Accept-Language':'en-US, en;q=0.5'})

home = requests.get(Website_url,headers=HEADERS)
home_soup = BeautifulSoup(home.content, 'lxml')

In [3]:
image_url = []
img_table = home_soup.find_all('div', class_='slick-slide')
img_table
for img in img_table:
    img_tag = img.find('img')
    # print(img_tag)

    url = img_tag.get('src')
    image_url.append(url)

image_url


['https://typewriter.imgix.net/u/68fed2ef-0e63-4b79-a96e-e80b38da4ecd/p/137025/three-boys-in-dock.jpg?ixlib=rails-2.1.4&auto=format%2Ccompress&crop=faces&fit=crop&h=675&w=900',
 'https://typewriter.imgix.net/u/68fed2ef-0e63-4b79-a96e-e80b38da4ecd/p/137028/DSC_7429.jpeg?ixlib=rails-2.1.4&auto=format%2Ccompress&crop=faces&fit=crop&h=675&w=900',
 'https://typewriter.imgix.net/u/68fed2ef-0e63-4b79-a96e-e80b38da4ecd/p/137035/downgreenday012.jpeg?ixlib=rails-2.1.4&auto=format%2Ccompress&crop=faces&fit=crop&h=675&w=900',
 'https://typewriter.imgix.net/u/68fed2ef-0e63-4b79-a96e-e80b38da4ecd/p/137031/LookingGlass32.jpeg?ixlib=rails-2.1.4&auto=format%2Ccompress&crop=faces&fit=crop&h=675&w=900',
 'https://typewriter.imgix.net/u/68fed2ef-0e63-4b79-a96e-e80b38da4ecd/p/137030/JJM_9261.jpeg?ixlib=rails-2.1.4&auto=format%2Ccompress&crop=faces&fit=crop&h=675&w=900',
 'https://typewriter.imgix.net/u/68fed2ef-0e63-4b79-a96e-e80b38da4ecd/p/137026/_JON7252.jpeg?ixlib=rails-2.1.4&auto=format%2Ccompress&crop

In [4]:
description=[]
desc1_table=home_soup.find_all('div',attrs={'class':'content-inner'})
for i in desc1_table:
    desc1=i.find('h1').text.strip()
    description.append(desc1)
    desc2=i.find('p').text.strip()
    description.append(desc2)
    
desc2_table = home_soup.find_all('div', attrs={'class': 'content col-sm-12'})
for j in desc2_table:
    p_tags = j.find_all('p')
    for p in p_tags:
        description.append(p.text.strip().replace('\u201c', '').replace('\u201d', '').replace('\u2019', ''))

In [10]:
Location='413 Mondamin Rd, Zirconia, NC'
Zip_code='28790'
Telephone=home_soup.find('a',attrs={'class':'phone'}).text.strip()
Mail='mondamin@mondamin.com'

# Location and Contact details in the desired format
location_data = {
    "address": Location,
    "zip_code": Zip_code,
    "latitude": '35.19663',  # You can replace this with actual latitude if needed
    "longitude": '-82.47956'  # You can replace this with actual longitude if needed
}

contact_data = {
    "mail": Mail,
    "telephone": Telephone
}
location_data
contact_data

{'mail': 'mondamin@mondamin.com', 'telephone': '(828) 693-7446'}

In [12]:
Socials=[]
Facebook = home_soup.find('a', attrs={'class': 'social facebook'}).get('href')
Socials.append({'facebook': Facebook})

# Add Pinterest link
pinterest = home_soup.find('a', attrs={'class': 'social pinterest'}).get('href')
Socials.append({'pinterest': pinterest})

# Add Instagram link
Instagram = home_soup.find('a', attrs={'class': 'social instagram'}).get('href')
Socials.append({'instagram': Instagram})

Socials

[{'facebook': 'https://www.facebook.com/Camps-Mondamin-and-Green-Cove-46724246730/'},
 {'pinterest': 'https://www.pinterest.com/campmondamin/'},
 {'instagram': 'https://www.instagram.com/campmondamin/'}]

In [13]:
import re
import requests
from bs4 import BeautifulSoup

dr_url = 'https://www.mondamin.com/2025-dates-rates'
dr = requests.get(dr_url, headers=HEADERS)
dr_soup = BeautifulSoup(dr.content, 'lxml')

Camp_Details = []
columns = dr_soup.find_all('div', class_='column-wrapper')

for column in columns:
    content = column.find('div', class_='column-content')
    if not content:
        continue 

    camp_names = content.find_all('h3')
    camp_details = content.find_all('p')

    for name, detail in zip(camp_names, camp_details):
        details_text = detail.text.strip()
        match = re.search(
            r'(?P<Duration>.+?)Ages (?P<Age>\d+–\d+)\s*(?P<Date>.+?)\$(?P<Fee>\d[\d,]*)',
            details_text
        )
        if match:
            # Extract dates and split into start_date and end_date
            date_text = match.group("Date").strip()
            date_split = re.split(r'\s*–\s*|\s+to\s+', date_text)
            start_date = date_split[0].strip()
            end_date = date_split[1].strip() if len(date_split) > 1 else ""

            # Fix age to ensure the dash is replaced with '-'
            age = match.group("Age").strip().replace('\u2013', '-')

            Camp_Details.append({
                "name": name.text.strip(),
                "duration": match.group("Duration").strip(),
                "eligible": age,
                "start_date": start_date,
                "end_date": end_date,
                "cost": f"${match.group('Fee').strip()}",
                "deposits":'null',
                'location':'null'
            })

Camp_Details


[{'name': 'Junior Camp',
  'duration': '5 Days',
  'eligible': '6-10',
  'start_date': 'Wed., 5/28',
  'end_date': 'Sun., 6/1',
  'cost': '$1,785',
  'deposits': 'null',
  'location': 'null'},
 {'name': 'June Camp',
  'duration': '3 Weeks',
  'eligible': '7-13',
  'start_date': 'Wed., 6/4',
  'end_date': 'Mon., 6/23',
  'cost': '$6,100',
  'deposits': 'null',
  'location': 'null'},
 {'name': 'Main Camp',
  'duration': '5 Weeks',
  'eligible': '8-17',
  'start_date': 'Fri., 6/27',
  'end_date': 'Fri., 8/1',
  'cost': '$8,600',
  'deposits': 'null',
  'location': 'null'},
 {'name': 'August Camp',
  'duration': '10 days',
  'eligible': '6-11',
  'start_date': 'Tues., 8/5',
  'end_date': 'Thurs, 8/14',
  'cost': '$3,570',
  'deposits': 'null',
  'location': 'null'}]

In [14]:
cat_url = 'https://www.mondamin.com/program/philosophy'
cat = requests.get(cat_url, headers=HEADERS)
cat_soup = BeautifulSoup(cat.content, 'lxml')

Categories=[]
category=cat_soup.find('div',attrs={'class':'content col-sm-11 col-md-10 col-lg-8'}).text.strip()
description=cat_soup.find('div',attrs={'class':'content col-sm-12'}).text.strip().replace('\n','')
img='https://typewriter.imgix.net/u/68fed2ef-0e63-4b79-a96e-e80b38da4ecd/p/21039/DSC_7692.JPG?auto=format'
Categories.append({
                "name": category,
                "description":description,
                "image":img
            })

In [15]:
# Define the URL and headers
act_url = 'https://www.mondamin.com/program/activities'
act = requests.get(act_url, headers=HEADERS)
act_soup = BeautifulSoup(act.content, 'lxml')

# Extract activities
Activities = []
activities_table = act_soup.find_all('div', attrs={'class': 'content col-sm-12'})

for section in activities_table:
    activity_name = section.find('h2')
    if activity_name:
        activity_name = activity_name.text.strip()
    
    activity_desc = section.find('p')
    if activity_desc:
        activity_desc = activity_desc.text.strip().replace('\u201c', '').replace('\u201d', '').replace('\u2019', '').replace('\u2013','').replace('\u2026','')

    if activity_name and activity_desc:
        Activities.append({
            'name': activity_name,
            'description': activity_desc
        })

# Extract images starting from the second one
image_urls = []
sections = act_soup.find_all('section', id=lambda x: x and x.startswith("block-id-"))
for section in sections:
    image_div = section.find('div', class_='container imgix-fluid-bg')
    if image_div and 'data-src' in image_div.attrs:
        image_urls.append(image_div['data-src'])

# Add images to Activities starting from the second image
for i, activity in enumerate(Activities):
    if i + 1 < len(image_urls):  # Ensure there are enough images
        activity['image_url'] = image_urls[i + 1]

# Display updated Activities
Activities


[{'name': 'Swimming',
  'description': 'Swimming is basic to safety on the waterfront, we want all campers to be at least at an intermediate level; those who are not take lessons at least once a day. Since ours is a non-competitive program, the emphasis is on form and endurance  not on speed. All swimming is supervised by lifeguards.',
  'image_url': 'https://typewriter.imgix.net/u/68fed2ef-0e63-4b79-a96e-e80b38da4ecd/p/19624/IMGP1174.jpg?fit=crop&w=1600&h=1067&crop=faces&auto=format'},
 {'name': 'Canoeing & Kayaking',
  'description': 'We begin on the lake, where we work on safety and basic strokes. When ready, campers take river trips, training at first on easy Class I and II rivers, then moving up in difficulty and challenge as they are able. We take easy river and lake trips for the inexperienced paddlers, and one to five day trips on Class III and IV rivers for those ready. We spend lots of time teaching how to read water, ferry, catch eddies, and play in the rivers many waves and

In [16]:
Mondamin= [{
    'name': Website_name,  
    'url': Website_url,
    'description':description,
    'location': location_data,
    'contact': contact_data,
    'socials': Socials,
    'image_urls':image_url,
    'sessions': Camp_Details,
    'categories': Categories,
    'activities': Activities
}]
file_path = 'mondamin_data.json'

# Export the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(Mondamin, json_file, indent=4)